In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
from MPyDATA_examples.Olesik_et_al_2020.analysis import compute_figure_data
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA_examples.Olesik_et_al_2020.plotter import Plotter
from MPyDATA_examples.Olesik_et_al_2020.convergence import convergence_in_time, convergence_in_space
from MPyDATA_examples.Olesik_et_al_2020.distributions import n_n, n_s
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2
from MPyDATA_examples.utils.error_norms import L2
from MPyDATA_examples.utils.show_plot import show_plot

import matplotlib.pyplot as plt
import numpy as np

In [3]:
psi_coord = x_p2()  # TODO: automate repetition over x_p2, x_ln, ....
results, setup = compute_figure_data(psi_coord=psi_coord)

In [4]:
for coord in results.keys():
    plotter = Plotter(setup)
    for opt_i, opts in enumerate(results[coord]['numerical'].keys()): 
        plot_data = results[coord]['numerical'][opts]
        for i in range(len(setup.nt)):
            mnorm = setup.mixing_ratios[i]
            t = setup.nt[i] * setup.dt
            if opt_i == 0:
                plotter.pdf_curve(PdfEvolver(setup.pdf, setup.drdt, t), mnorm)
                plotter.pdf_histogram(
                    results[coord]['grid']['r'], 
                    results[coord]['analytical'][i], 
                    bin_boundaries = results[coord]['grid']['rh'], 
                    label='discretised', 
                    mnorm=mnorm,
                    color='red'
                )
            plotter.pdf_histogram(
                results[coord]['grid']['r'], 
                plot_data[i], 
                label=opts, 
                bin_boundaries=results[coord]['grid']['rh'], 
                mnorm=mnorm
            )
    print(f"grid_layout: {coord}")
    show_plot()

grid_layout: x_id


grid_layout: x_p2


grid_layout: x_ln


In [5]:
def convergence_in_space(nr: list, results):
    err_data = {}
    for coord in results.keys():
        err_data[coord] = {}
        for opts in results[coord]["error_L2"].keys():
            err_data[coord][opts] = {'dx':[], 'error_L2':[]} 

    for nri, nr in enumerate(nrs):
        data, setup = compute_figure_data(nr=nr, psi_coord=psi_coord)
        for coord in data.keys():
            for opts in data[coord]["error_L2"].keys():
                err_data[coord][opts]['dx'].append(data[coord]["grid"]['dx'])
                err_data[coord][opts]['error_L2'].append(data[coord]["error_L2"][opts])
    return err_data

In [6]:
nrs = [32, 64]  #TODO: do not repeat calc. for default nr!
err_data_nr = convergence_in_space(nr=nrs, results=results)

In [7]:
for coord in results.keys():
    for i, opts in enumerate(err_data_nr[coord].keys()):
        x = np.log2(err_data_nr[coord][opts]['dx'])
        y = err_data_nr[coord][opts]['error_L2']
        plt.plot(x, y, label=opts, linestyle=':')
        plt.scatter(x, y)
    plt.title(f"grid layout: {coord}")
    
    for k in [2,3]:
        plt.plot(x, k*x+0, '--', label=f"error ∝ $Δx^{k}$", color='black')
    
    plt.legend()
    plt.gca().set_xlabel('log$_2$(dx)')
    plt.gca().set_ylabel('log$_2(err_{L2})$')
    plt.grid()
    show_plot()

In [8]:
def convergence_in_time(dt: list, results):
    err_data = {}
    for coord in results.keys():
        err_data[coord] = {}
        for opts in results[coord]["error_L2"].keys():
            err_data[coord][opts] = []

    for dti, dt in enumerate(dts):
        data, setup = compute_figure_data(dt=dt, psi_coord=psi_coord)
        for coord in data.keys():
            for opts in data[coord]["error_L2"].keys():
                err_data[coord][opts].append(data[coord]["error_L2"][opts])
    return err_data

In [9]:
dts = [.25, .125]
err_data_dt = convergence_in_time(dt = dts, results = results)

In [10]:
x = np.log2(dts)
for coord in results.keys():
    for i, opts in enumerate(err_data_dt[coord].keys()):
        plt.plot(x, err_data_dt[coord][opts], label=opts, linestyle=':')
        plt.scatter(x, err_data_dt[coord][opts])
    plt.title(coord)
    plt.legend()
    
    for k in [2,3]:
        plt.plot(x, k*x-0.5, ':', label=f"error ∝ $Δx^{k}$", color='black')
    
    plt.gca().set_xlabel('log$_2$(dt)')
    plt.gca().set_ylabel('log$_2(err_{L2})$')
    plt.grid()

    show_plot()